In [1]:
#CSV manage files
import csv
import xlsxwriter
#Provee un conjunto de estructuras para manipular datos
import pandas as pd
#Posee herramientas para el procesamiento de lenguaje natural
import nltk
from nltk.probability import FreqDist #Frecuencia de las palabras
from nltk.tokenize import word_tokenize #Hace la tokenización de un texto
from nltk import tokenize
#Obtenemos las stopwords
from nltk.corpus import stopwords
#Para trabajar con el clasificador
from nltk.classify import NaiveBayesClassifier
#Para obtener todos los signos de puntuación
import string
import re #Regex

In [2]:
datos_excel = pd.read_excel(open('dbUniversidadesEtiq3282.xlsx','rb'))
datos_excel = pd.DataFrame(datos_excel)
datos_excel.head()

,Tweet,Retweets,Favoritos,Sentimiento
0,Buen día. En estos momentos estamos de vacaci...,0,0,neu
1,El Museo de Arte Colonial Quinta Anauco @Quint...,5,6,pos
2,#FelizJueves #29Ago #EstudiaaDistanciaenlaUNA ...,5,9,neu
3,RT @educacion3_0: Recopilación de las mejores ...,33,0,pos
4,RT @jornadasuna: #FelizLunes #26agosto ya se e...,9,0,neu


In [3]:
sample = datos_excel.sample(len(datos_excel))
sample

,Tweet,Retweets,Favoritos,Sentimiento
358,RT @LicGardeazabal: 1 de abril día mundial de ...,12,0,pos
2733,RT @VTVcanal8: Venezolanos se concentran en pl...,19,0,neu
1905,Aprende a diseñar una estructura de costos con...,2,6,neu
1845,RT @ALTEC2018: Actividades Pre-Evento #AltecVE...,5,0,neu
1141,Conformado Grupo de Investigación en Biotecnol...,3,3,pos
1501,"@paola_1402 Apenas nos informen, lo publicamos",0,0,neu
841,RT @UNETinvestiga: Hoy inicia la #2daEtapa de ...,9,0,neu
1770,@deylisloyo @trafficLARA @uclaedumedia @opsuof...,0,1,neu
1935,El Día de la Resistencia Indígena fue original...,3,9,pos
2008,RT @ameliasequera: La @UCLAve está de Aniversa...,4,0,pos


In [4]:
palabrasInutiles = stopwords.words("spanish") + list(string.punctuation) + ['https','“','”','¡',"¿",'día',"''",'``','//','...','http','…','this','flipbook','out','check','aún','N°','dr.','rt']

In [5]:
def construirBolsaDePalabras(palabras):
    diccionario={}
    for palabra in word_tokenize(palabras):
        #print(palabra)
        if palabra not in palabrasInutiles:
            diccionario[palabra]=1
    #print(palabra)
    return diccionario

In [6]:
rasgosNegativos = []
rasgosPositivos = []
rasgosNeutrales = []
#for row in sample['Tweet']:
	#rasgosNegativos.append((construirBolsaDePalabras(row[2]),row[5]))
    #print(row)
    
for indice_fila, fila in sample.iterrows():
    #print(fila['Tweet'])
    tweet = fila['Tweet'].lower()
    tweet = re.sub(r'@[a-zA-Z0-9-_.]+', '', tweet)
    tweet = re.sub(r'#[a-zA-Z0-9-_.]+', '', tweet)
    tweet = re.sub(r'https://[a-zA-Z0-9-_./]+', '', tweet)
    tweet = re.sub(r'¡', '', tweet)
    tweet = re.sub(r'[a-zA-Z0-9]+[…]', '', tweet)
    if fila['Sentimiento'] == 'pos':
        #rasgosPositivos.append((construirBolsaDePalabras(fila['Tweet']),fila['Sentimiento']))
        rasgosPositivos.append((construirBolsaDePalabras(tweet),fila['Sentimiento']))
    elif fila['Sentimiento'] == 'neu':
        #rasgosNeutrales.append((construirBolsaDePalabras(fila['Tweet']),fila['Sentimiento']))
        rasgosNeutrales.append((construirBolsaDePalabras(tweet),fila['Sentimiento']))
    else:
        #rasgosNegativos.append((construirBolsaDePalabras(fila['Tweet']),fila['Sentimiento']))
        rasgosNegativos.append((construirBolsaDePalabras(tweet),fila['Sentimiento']))

In [7]:
len(rasgosPositivos)

1378

In [8]:
len(rasgosNeutrales)

1576

In [9]:
len(rasgosNegativos)

327

In [52]:
int(len(rasgosNegativos)*0.95)

192

In [10]:
rasgosPositivos[26]

({'consultas': 1,
  'pediátricas': 1,
  'ofrecidas': 1,
  'jornada': 1,
  'salud': 1,
  'fundación': 1,
  'nawaraos': 1,
  'ucla': 1,
  'todavía': 1},
 'pos')

In [18]:
#Para limitar la cantidad de comentarios que estarán en la sesión de entrenamiento
divP=int(len(rasgosPositivos)*0.95)#0.75
divNeu=int(len(rasgosNeutrales)*0.95)
divNeg=int(len(rasgosNegativos)*0.95)
clasificadorSentimiento=NaiveBayesClassifier.train(rasgosPositivos[:divP]+rasgosNegativos[:divNeg]+rasgosNeutrales[:divNeu])

In [19]:
#Precisión del modelo con los mismos datos que se utilizaron en el entrenamiento
nltk.classify.util.accuracy(clasificadorSentimiento,rasgosPositivos[:divP]+rasgosNegativos[:divNeg]+rasgosNeutrales[:divNeu])

0.7750320924261874

In [20]:
#Precisión del modelo con los datos de prueba
nltk.classify.util.accuracy(clasificadorSentimiento,rasgosPositivos[divP:]+rasgosNegativos[divNeg:]+rasgosNeutrales[divNeu:])

0.6303030303030303

In [21]:
clasificadorSentimiento.show_most_informative_features()

Most Informative Features
           fallecimiento = 1                 neg : pos    =     71.6 : 1.0
                   debes = 1                 neg : pos    =     26.7 : 1.0
               rectorado = 1                 neg : pos    =     26.7 : 1.0
                  afecta = 1                 neg : neu    =     24.1 : 1.0
                    dice = 1                 pos : neu    =     21.7 : 1.0
                   medio = 1                 neg : pos    =     18.3 : 1.0
                gobierno = 1                 neg : pos    =     18.3 : 1.0
                    debe = 1                 neg : pos    =     18.3 : 1.0
                 pastora = 1                 pos : neu    =     17.9 : 1.0
                felicita = 1                 pos : neu    =     15.6 : 1.0


## Prueba

In [30]:
tweet = "no tengas feliz de renunciar a lo bueno, para perseguir lo grandioso: jhon d. rockefeller"
#tweet="RT @CampusCaracas: Ya están a punto de viajar"
print(tweet)
tweet = tweet.lower()
tweet = re.sub(r'@[a-zA-Z0-9-_.]+', '', tweet)
tweet = re.sub(r'#[a-zA-Z0-9-_.]+', '', tweet)
tweet = re.sub(r'https://[a-zA-Z0-9-_./]+', '', tweet)
tweet = re.sub(r'¡', '', tweet)
tweet = re.sub(r'[a-zA-Z0-9]+[…]', '', tweet)
#textoTokenizado=tokenize.casual_tokenize(tweet)
bolsa=construirBolsaDePalabras(tweet)
print(bolsa)

no tengas feliz de renunciar a lo bueno, para perseguir lo grandioso: jhon d. rockefeller
{'feliz': 1, 'renunciar': 1, 'bueno': 1, 'perseguir': 1, 'grandioso': 1, 'jhon': 1, 'd.': 1, 'rockefeller': 1}


In [31]:
clasificadorSentimiento.classify(bolsa)

'pos'

In [233]:
#Cuenta cuantas coincidencias de una palabra existen en las listas
count = 0
for Utuple in rasgosNegativos:
    count = count + list(Utuple[0]).count('pereira')
count

4

## Clasifica los tweets restantes del excel

In [32]:
datos_excel = pd.read_excel(open('dbUniversidadesSinEtiquetar.xlsx','rb'))
datos_excel = pd.DataFrame(datos_excel)
datos_excel.head()

,Tweet,Retweets,Favoritos,Sentimiento
0,RT @AncaDigital: Son 56 años al servicio de ce...,11,0,NaN
1,RT @Radiominuto790: @UCLAve 56 años dando la b...,4,0,NaN
2,RT @martha_astral: @UCLAve Hoy de aniversario ...,2,0,NaN
3,RT @lusolarenseac: Van 56 años dando la bienve...,10,0,NaN
4,RT @miguel_e_: Que lindura orgulloso de ser Uc...,2,0,NaN


In [33]:
sample = datos_excel.sample(len(datos_excel))
sample

,Tweet,Retweets,Favoritos,Sentimiento
412,RT @contrapuntovzla: Con motivo de la celebrac...,6,0,NaN
4679,La UNEG fija posición en cuanto al informe emi...,6,5,NaN
5983,#SaberesUNERG Unerg conversa sobre la Constru...,4,2,NaN
6965,RT @VTVcanal8: Movimientos sociales del mundo ...,37,0,NaN
13143,"Sé parte de la lucha, multiplicadora de la pre...",0,1,NaN
13839,¡INSCRIPCIONES ABIERTAS A TODO PÚBLICO! \nCons...,0,0,NaN
3428,RT @vladimirpadrino: Con un interesante Ejerci...,1328,0,NaN
10278,La Universidad Privada Dr. Rafael Belloso Chac...,0,4,NaN
6069,RT @NicolasMaduro: Conmemoramos 241 años del n...,2636,0,NaN
5275,Tal día como hoy pero en el año 1917 nació Jua...,1,2,NaN


In [34]:
libro = xlsxwriter.Workbook('dbUniversidadesEtiqXClassfier.xlsx')
hoja = libro.add_worksheet('Data')

#myFile = open('dbUniversidades.csv', 'w')
data = ["Tweet", "Retweets", "Favoritos", "Sentimiento"]
row = 0
col = 0
 
# Iteramos los datos para ir pintando los encabezados
for titulo in data:
    hoja.write(row, col, titulo)
    col += 1
row = 1
col = 0

for indice_fila, fila in sample.iterrows():
    print(fila[0])
    tweet = fila[0].lower()
    tweet = re.sub(r'@[a-zA-Z0-9-_.]+', '', tweet)
    tweet = re.sub(r'#[a-zA-Z0-9-_.]+', '', tweet)
    tweet = re.sub(r'https://[a-zA-Z0-9-_./]+', '', tweet)
    tweet = re.sub(r'¡', '', tweet)
    tweet = re.sub(r'[a-zA-Z0-9]+[…]', '', tweet)
    bolsa=construirBolsaDePalabras(tweet)
    sent = clasificadorSentimiento.classify(bolsa)
    print(sent)
    hoja.write(indice_fila+row, 0, fila[0])
    hoja.write(indice_fila+row, 1, fila[1])
    hoja.write(indice_fila+row, 2, fila[2])
    hoja.write(indice_fila+row, 3, sent)
libro.close()

RT @contrapuntovzla: Con motivo de la celebración del Día de Portugal, lo mejor de la cultura musical lusitana se dejará ver de forma gratu…
pos
La UNEG fija posición en cuanto al informe emitido por la Alta Comisionada Michelle Bachelet ante el Consejo de Der… https://t.co/qXvJEi884Y
neg
#SaberesUNERG Unerg conversa sobre la  Construcción Permanente y el Fortalecimiento Socio-productivo y sus efectos… https://t.co/7jguZi0AUR
neg
RT @VTVcanal8: Movimientos sociales del mundo repudian intento de golpe de Estado en Venezuela 

https://t.co/uu3wap8hxJ

#TrabajadoresPorL…
neg
Sé parte de la lucha, multiplicadora de la prevención
#tocate #octubrerosa #Psm https://t.co/76M1YtNqkC
neg
¡INSCRIPCIONES ABIERTAS A TODO PÚBLICO! 
Consulta costos sólo vía mensaje privado o en la Sede Diplomados Centro 
.… https://t.co/a4psyaaJnh
neg
RT @vladimirpadrino: Con un interesante Ejercicio Militar, invitado por el Ministro de Defensa Turco General Hulusi AKAR finalicé mi agenda…
neg
La Universidad Privada 

Lapso de consignación de documentos y solvencias para el Acto Académico XL

El lapso para revisión y entrega de doc… https://t.co/atWKAnbTte
neg
Lee o descarga en https://t.co/oI1DhPo5qm  la edición Nro.17 de Strategos, publicación científica arbitrada e index… https://t.co/uQC5snGrP0
pos
Síguenos en Instagram @Monteavila
neu
Buenas Tardes! el proceso de inscripción se realiza en 6 pasos. A partir de jueves  10 de enero hay atención  en la… https://t.co/Nf1l5RyZOi
neu
RT @MIJPVenezuela: NOTICIA 👉 Incautaciones de marihuana en Venezuela representan más del 80% en el 2019 (Más Información) https://t.co/CLJr…
neg
#17Mayo Celebramos #DíaMundialdelReciclado, 
El objetivo de esta #Conmemoración es promover el consumo de artículos… https://t.co/KazjGROVOz
pos
Nuestra escuela @DANZALUZ está de aniversario y lo celebraremos en el @TeatroBaralt con la presentación del espectá… https://t.co/WvcyTkCSWS
pos
#UCV_Noticias El Proyecto de Ley de Amnistía está bajo el escrutinio de los expertos - Por @

RT @metro_caracas: #ATENCION se informa a nuestros usuarios que motivado a falla de energía eléctrica en estos momentos el sistema #MetroDe…
neg
#UCeVistas MUJERES HAN ALCANZADO PAPELES PROTAGÓNICOS EN LA PALESTRA POLÍTICA MUNDIAL #UCV #UCVNoticias https://t.co/30iGEYOOZI
pos
Hoy se cumplen 47 años de la creación del Instituto Universitario Politécnico de Guayana (IUPG) , hoy Vicerrectorad… https://t.co/ZWQv0CRUXt
neg
RT @lafuerzadchavez: 4/4 Poco a poco iremos recuperando nuestro sistema eléctrico y generando mayor distribución de energía hacia el resto…
neg
Buenos días ¡Feliz inicio de semana!☀️🙌
pos
¿Recuerdas nuestro festival del talento el Pingüino? Actualiza tus datos https://t.co/H792kkXDv2 #egresados
neu
Hoy #24May se conmemora el #DíaDelTelegrafista en #Venezuela para rememorar que en 1844 el catedrático de dibujo, S… https://t.co/grhQFECbnM
pos
RT @NestorReverol: ¡FELICITACIONES BOMBEROS DE LA PATRIA! Desde la Academia Militar hemos certificado a 717 nuevos bomberos a través 

neu
La Secretaría General informa a los bachilleres asignados por OPSU-CNU 2018, que a partir del 24 de septiembre al 1… https://t.co/r1zehywlDN
neu
RT @NestorReverol: Nuestro Pdte @NicolasMaduro ha certificado a 8 mil 901 guardaparques y bomberos forestales. Además aprobó recursos para…
neg
Representantes de sindicatos nacionales de trabajadores universitarios visitaron sedes de la Unermb… https://t.co/MhGlDtTwWD
neg
Primer Taller de Empoderamiento de Mujeres Claves #ReconciliACCIÓN @OportunidadAC @MujerCiudadania https://t.co/vcPumCky2i
pos
#Atención se le informa a los graduandos de #ElCallao y #Guasipati que las fechas para el ensayo, firma de acta y a… https://t.co/IcD7PpgMeK
neu
RT @MIJPVenezuela: EN VIDEO 👉🏽 Ministro @NestorReverol firma actas desde el Mauseleo del Libertador como parte de los actos conmemorativos…
neg
#BrevesUneg Uneg se mantiene creando lazos productivos para proyecto Protepui https://t.co/ot9qABsl3y  #UNEG… https://t.co/H84XWiGEYz
neg
RT @INDeporteVE: ¡COMIEN

RT @DIGAES_UC: https://t.co/ExNpu0VCAk EL PROCESO PARA SOLICITUDES HISTORIAL ACADÉMICO CERTIFICADO ESTARÁ ACTIVO SEGÚN EL SIGUIENTE CALENDA…
neu
Se le informa a los estudiantes regulares de 2do, 3ro y 4to año Periodo I-2019 de la Escuela de Enfermería (Licenci… https://t.co/Ctn3EfoSHr
neu
RT @NicolasMaduro: El 18° CLAE Venezuela 2019 permanecerá vivo en los debates de la juventud desde los espacios donde hacen vida, en cada u…
pos
#Atención Ya está abierto el proceso de preinscripción para las maestrías en Especialización en Gerencia, mención… https://t.co/wpIPqWcGRe
pos
RT @avnve: #PoderPopular || Maduro: Estabilidad política, protección social y crecimiento económico son la base para frenar ataques imperia…
neg
RT @vladimirpadrino: Pedir a la FANB salir del carril constitucional y desconocer sus autoridades es un acto irresponsable y sería llevar a…
neg
RT @UCV_Noticias: La Coordinación de Extensión de la Facultad de Ciencias @ciencia_infoUCV y la agrupación @CienciasEsRitmo invitan 

...

neg
RT @avnve: #Economia || Gobierno Nacional inicia siembra de cereales en llanos venezolanos

➡ https://t.co/9Xm16C9Ut2 https://t.co/4EpexEGw…
neg
RT @Mppeuct: Líderes Estudiantiles en el II Congreso de la Federeción de Estudiantes Universitarios
 https://t.co/Ekfri30zMB
neu
#atención estudiantes! Ya se encuentran abiertas las inscripciones para el Trimestre Sep-Dec 2019 , Para informació… https://t.co/WuvsLEJTTv
pos
#14Dic // 48 Ingenieros Marítimos y 11 T.S.U. en Transporte Acuático son los protagonistas del Acto de Grado que en… https://t.co/fxanM4qNkL
pos
Felicidades a los 235 graduandos de la sede #sanfrancisco #unermb en #pregrado y #pregrado @SeppeuctiGBZ @Mppeuct… https://t.co/YBB8y3CLi1
pos
CU aprobó ampliación del monto de cobertura de seguro HCM-SAMHOI. https://t.co/dRibTpCrKB https://t.co/eai9XO7kOM
pos
#FrasesUCV_Noticias Todo aquel que haya hecho algo importante en la vida, fue disciplinado: Andrew Hendrixson #UCV https://t.co/Pri0Hlxy2h
pos
Los recordamos siempre =) 

Feliz día Nacional del Transportista a todos nuestros profesionales del volante que con trabajo, esfuerzo y dedicac… https://t.co/obCDOX9FMF
pos
Recordamos que el Miércoles 24 de abril corresponde la inscripción a Derecho y Diseño Gráfico, el 25 a Humanidades… https://t.co/qr9d8ImqVZ
neu
RT @jlberroteran: #29Mar nuestra tribuna antiimperialista en la Casona, debatiendo sobre los ataques a la estabilidad social, han querido p…
neg
Liderazgo ucevista firme en el respeto a la institucionalidad - Por @hluque  Fotografía @XiobelRomero… https://t.co/4nFcXdXnO9
pos
#UCeVistas esto es para ti @empresaspolar esta en la búsqueda de #Pasantes en el área de #administración… https://t.co/qBSi7CQLGE
neu
#Atención Aquí está la lista de los graduandos de la II Promoción 2018 #DraCarmenKolosovas en #CiudadBolívar #UNEG… https://t.co/Fdr77jskqa
neu
Bajo el lineamiento del rector @jlberroteran cuya gestión apunta a una universidad productiva y popular, y a la inv… https://t.co/e84HtrXJNg
neg
Durante el a

neg
#FrasesUCV_Noticias #UCeVistas Una persona debe fijar sus objetivos cuanto antes y dedicar toda su energía y talento a ellos: Walt Disney
neg
Entregan certificados  del Diplomado en Sistema de Gestión de la Calidad https://t.co/soJFm1RLQo https://t.co/fVODTY8kNs
pos
🔝Horario de atención de la Coordinación de Pasantías Facultad de Derecho. A todos aquellos estudiantes que no han r… https://t.co/xqkzDBjypT
pos
1.- Te invitamos a la charla Cruz Diez y su Arte. 
Fecha: Jueves, 08 de agosto de 2019.
Hora: 10:00 am
Lugar: Sala… https://t.co/zcl5mLwYOK
pos
#NoALaInjerenciaSiALaPaz ni a través de ayuda humanitaria, ni a través de guerras #VenezuelaFirmaPorLaPaz… https://t.co/HlVirCYyMP
pos
Qué esperas? Inscripciones abiertas para Nuevo Ingreso,  Regulares y Reincoporados. El  Inicio de Semestre 2019-1 e… https://t.co/cesS8ZzxUq
pos
@wladbric Hola, Wladimir. Al definirse las nuevas fechas del Calendario de Grado, lo informáremos. Los directivos i… https://t.co/nDlmE3Pvi0
neg
Acaba de publ

pos
RT @VTVcanal8: Venezuela conmemora 115 años del natalicio de Pablo Neruda

#DiplomaciaDePaz

https://t.co/yBQKo652hZ
pos
RT @FacIngUniojeda: ¡Te invitamos cordialmente a participar en las actividades planificadas para celebrar el XVI Aniversario de @UNIOJEDA y…
pos
Autoridades plantean el logro de una seguridad social integral para el profesorado https://t.co/orPbwNWeIP https://t.co/EXqNNY86tv
neg
Ya están abiertas las preinscripciones para nuevos aspirantes.
Costo de la preinscripción Bs.30.000.… https://t.co/UIn4DZTvO4
neu
#BrevesUneg La Uneg realizó satisfactoriamente taller de edición de libros digitales #UNEG #SomosUneg… https://t.co/JsqXn0govf
pos
RT @meteorologiaumc: Miercoles, 06 de Febrero. Poca nubosidad durante la mañana. Parcialmente nublado al final de la tarde. Viento: ENE 11-…
neu
Hoy queremos honrar a todos los maestros en su día, pues con compromiso y sacrificio se empeñan en transformar al p… https://t.co/NWWJs8D9Et
neg
El Auditorio "Dr. Emilio Medina" de la prest

pos
RT @PDVSA: 🇻🇪👨‍🚒#SaberPetrolero Te invitamos a conocer sobre el Mejoramiento del Crudo Extrapesado ➙Un punto de partida para la formación d…
pos
¿Ya viste la segunda edición del micro informativo de NotiUnexpo?
.
.
Recuerden que pueden verlo a través de nuestr… https://t.co/az8qFIrOgC
neu
RT @teleSURtv: Sin los bosques amazónicos perderíamos gran parte de la producción de oxígeno y agua dulce del planeta

Lamentablemente el i…
pos
La Universidad Nacional Experimental de Guayana cumple 37 años de servicio. Queremos invitarte a compartir el logo… https://t.co/kCXLdzs8AU
neg
#CU #UCV, durante la sesión efectuada este #6Feb en el espíritu de colaborar en la orientación de la vida del país,… https://t.co/jgxiXqIJ80
pos
RT @FAPUV: Porcentaje de ajuste por escalafón, según tabla salarial enviada a las #UniVe, impuesta por el Ejecutivo Nacional. Hambre induci…
neg
RT @teleSURtv: Presidente @NicolasMaduro reconoce labor de activistas que defendieron Embajada venezolana  en #Washington 🇺🇸 👉 